# 1- Imports

In [63]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select

from os import getcwd, remove
# from pdb import set_trace as st
from pdb import set_trace as st

from time import sleep

from glob import glob

from zipfile import ZipFile

import pandas as pd

from IPython.display import display
pd.options.display.max_columns = None

import yfinance as yf

from datetime import datetime

# from bokeh.io import output_notebook, show
# from bokeh.plotting import figure
# from bokeh.models import ColumnDataSource
# output_notebook()

import plotly.express as px
import plotly.graph_objects as go

import pickle

In [64]:
def save_object(obj, file_name):
    dbfile = open(file_name, 'wb')
    pickle.dump(obj, dbfile)
    dbfile.close() 

def loadData(file_name): 
    dbfile = open(file_name, 'rb')
    obj = pickle.load(dbfile)
    dbfile.close()
    return obj

# 2- Retreiving data

In [65]:
def initate_browser():
    # config for now showing the download popup and setting the download dir
    
    # url = "https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=258"
    
#     data for US carriers, demosetic and international
    url = "https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=308"
    profile = webdriver.FirefoxProfile()
    profile.set_preference('browser.download.folderList', 2) # custom location
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.download.manager.showAlertOnComplete", False)
    profile.set_preference('browser.helperApps.neverAsk.saveToDisk', 'application/zip, text/html, application/x-zip-compressed')
    profile.set_preference("browser.helperApps.neverAsk.openFile",'application/zip, text/html, application/x-zip-compressed')
    profile.set_preference('browser.download.dir', getcwd())

    browser = webdriver.Firefox(profile)
    browser.get(url)
    
    #set AllVars's tickbox
    browser.find_element_by_css_selector('#AllVars').click()

    return browser

In [66]:
def download_file_year(browser, year):
    select = Select(browser.find_element_by_css_selector('#XYEAR'))
    select.select_by_value(str(year))
    # st()
    browser.find_element_by_css_selector('.tsbutton').click()

In [67]:
def check_if_file_are_downloaded():
    return len(glob("*.zip")) == len(range(1990, 2020))

In [68]:
def check_if_final_csv_is_made():
    return  bool(glob("data.csv"))

In [69]:
def download_files():
    browser = initate_browser()
    for year in range(1990, 2020):
        download_file_year(browser, year)
        sleep(30)
# download_file_year(browser, 2001)


In [70]:
def concatinate_files():
    list_of_dfs = []
    for file_name in glob("*.zip"):
        zipObj = ZipFile(file_name, "r")
        zipObj.extractall()
        csv_file_name = glob("*.csv")[0]
        temp_df = pd.read_csv(csv_file_name)
        list_of_dfs.append(temp_df)
        remove(csv_file_name)
#         print("z")

    full_df = pd.concat(list_of_dfs, axis=0)
    full_df.to_csv("data.csv")

In [71]:
def remove_extra_zip_files():
    for file in glob("*.zip"):
        remove(file)

In [72]:
def get_data():
    # check if final concatinated data.csv is already made
    if not check_if_final_csv_is_made():
        # check if zip files are downloaded
        if not check_if_file_are_downloaded():
            initate_browser()
            download_files()
            # st()
        concatinate_files()
        remove_extra_zip_files()

In [73]:
get_data()

# 3- Cleaning data

## 3-1 selecting columns, correcting dtypes, removing nulls

In [74]:
if "df" not in locals():
    df = pd.read_csv("./data.csv"        )

KeyboardInterrupt: 

In [13]:
needed_columns = [
    "UNIQUE_CARRIER_NAME",
    "CARRIER_NAME",
    "YEAR",
    "MONTH",
    "QUARTER",
    "PASSENGERS",
    "FREIGHT",
    "MAIL",
    "DISTANCE",
]

In [14]:
if "sub_df" not in locals():
    sub_df = df.copy()
    del df
    sub_df = sub_df[needed_columns]
sub_df

,UNIQUE_CARRIER_NAME,CARRIER_NAME,YEAR,MONTH,QUARTER,PASSENGERS,FREIGHT,MAIL,DISTANCE
0,Air Wisconsin Airlines Corp,Air Wisconsin Inc.,1992,1,1,0.0,0.0,0.0,76.0
1,Trans States Airlines,Trans States Airlines,1992,1,1,0.0,0.0,0.0,58.0
2,United Air Lines Inc.,United Air Lines Inc.,1992,1,1,0.0,0.0,0.0,1039.0
3,America West Airlines Inc.,America West Airlines Inc.,1992,1,1,0.0,0.0,0.0,239.0
4,America West Airlines Inc.,America West Airlines Inc.,1992,1,1,0.0,0.0,592.0,69.0
...,...,...,...,...,...,...,...,...,...
7541333,United Air Lines Inc.,United Air Lines Inc.,1990,7,3,60459.0,539207.0,325416.0,901.0
7541334,Southwest Airlines Co.,Southwest Airlines Co.,1990,8,3,60511.0,70918.0,0.0,239.0
7541335,United Air Lines Inc.,United Air Lines Inc.,1990,8,3,61702.0,737858.0,400101.0,901.0
7541336,United Air Lines Inc.,United Air Lines Inc.,1990,8,3,62332.0,2455148.0,280308.0,1846.0


In [15]:
# correcting the dtypes

sub_df = sub_df.astype(
    {
        "PASSENGERS": "int", 
        "FREIGHT": "int", 
        "MAIL": "int", 
        "DISTANCE": "int"
    }
)

In [16]:
# checking for null values

sub_df.isna().sum()

UNIQUE_CARRIER_NAME    1141
CARRIER_NAME           1141
YEAR                      0
MONTH                     0
QUARTER                   0
PASSENGERS                0
FREIGHT                   0
MAIL                      0
DISTANCE                  0
dtype: int64

In [17]:
sub_df = sub_df.dropna(axis=0)

# be sure that there are no nans
assert sub_df.isna().sum().sum() == 0

## 3-2 removing outliers

In [18]:
grouped_carrier = sub_df.groupby("UNIQUE_CARRIER_NAME").count()["YEAR"]


In [19]:
# keeping only carriers with more than 10000 total flights
list_of_valid_carriers = grouped_carrier[grouped_carrier > 10000].index.to_numpy()
sub_df = sub_df[sub_df["UNIQUE_CARRIER_NAME"].isin(list_of_valid_carriers)]

In [20]:
# making 3 features for regression
sub_df["PASSENGERS_x_DISTANCE"] = sub_df["PASSENGERS"] * sub_df["DISTANCE"]
sub_df["FREIGHT_x_DISTANCE"] = sub_df["FREIGHT"] * sub_df["DISTANCE"]
sub_df["MAIL_x_DISTANCE"] = sub_df["MAIL"] * sub_df["DISTANCE"]


In [21]:
sub_df

,UNIQUE_CARRIER_NAME,CARRIER_NAME,YEAR,MONTH,QUARTER,PASSENGERS,FREIGHT,MAIL,DISTANCE,PASSENGERS_x_DISTANCE,FREIGHT_x_DISTANCE,MAIL_x_DISTANCE
0,Air Wisconsin Airlines Corp,Air Wisconsin Inc.,1992,1,1,0,0,0,76,0,0,0
1,Trans States Airlines,Trans States Airlines,1992,1,1,0,0,0,58,0,0,0
2,United Air Lines Inc.,United Air Lines Inc.,1992,1,1,0,0,0,1039,0,0,0
3,America West Airlines Inc.,America West Airlines Inc.,1992,1,1,0,0,0,239,0,0,0
4,America West Airlines Inc.,America West Airlines Inc.,1992,1,1,0,0,592,69,0,0,40848
...,...,...,...,...,...,...,...,...,...,...,...,...
7541333,United Air Lines Inc.,United Air Lines Inc.,1990,7,3,60459,539207,325416,901,54473559,485825507,293199816
7541334,Southwest Airlines Co.,Southwest Airlines Co.,1990,8,3,60511,70918,0,239,14462129,16949402,0
7541335,United Air Lines Inc.,United Air Lines Inc.,1990,8,3,61702,737858,400101,901,55593502,664810058,360491001
7541336,United Air Lines Inc.,United Air Lines Inc.,1990,8,3,62332,2455148,280308,1846,115064872,4532203208,517448568


In [22]:
# tt_df["date"] = 
# "{}-{}-01".format(sub_df["YEAR"].astype("str"), sub_df["MONTH"].astype("str"))

In [23]:
# grouped_carrier_year_month = sub_df.groupby(["UNIQUE_CARRIER_NAME", "YEAR", "MONTH"]).count()


In [24]:
# grouped_carrier_year_month

In [25]:
# tt = sub_df.groupby(["UNIQUE_CARRIER_NAME", "YEAR", "MONTH"]).count()

In [26]:
# tt

I just realized that only a couple of these airlines are traded publicly, so there is no need to remove the outliers furthuremore. I will just join them with a table that has all the publicly traded airlines.

# 4- Running a model for a single airline

## 4-1 Getting one single stock and cleaning the data

In [27]:
# one of the publicly traded airlines is "American Airlines Inc."
# with stock symbol of "AAL"

In [28]:
# downloading aal data

aal = yf.download(
    tickers = "AAL",
    period = "max",
    interval = "1mo",
    auto_adjust = True,
    threads = True,
    actions=True
)

[*********************100%***********************]  1 of 1 completed


In [29]:
aal.isna().sum()

Open            24
High            24
Low             24
Close           24
Volume          24
Dividends        0
Stock Splits     0
dtype: int64

In [30]:
# removing nulls
aal = aal.dropna(axis=0)

In [31]:
aal

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2005-09-01,19.844708,20.174668,18.006363,19.806999,1.091060e+07,0.0,0.0
2005-10-01,19.703296,23.898496,19.656159,23.266859,1.919910e+07,0.0,0.0
2005-11-01,23.285711,32.571712,23.276284,31.666681,1.940250e+07,0.0,0.0
2005-12-01,32.053212,36.578371,29.583229,35.013420,1.061080e+07,0.0,0.0
2006-01-01,35.069980,38.275299,26.679580,27.660032,1.776520e+07,0.0,0.0
...,...,...,...,...,...,...,...
2019-12-01,28.683999,29.700242,26.581768,28.574404,1.338269e+08,0.0,0.0
2020-01-01,28.873299,29.192120,25.276592,26.741179,2.267056e+08,0.0,0.0
2020-02-01,26.830846,30.666671,18.700891,18.979858,2.629806e+08,0.0,0.0


In [32]:
aal["Year"] = aal.index.year
aal["Month"] = aal.index.month
aal_needed = aal[["Close", "Year", "Month"]]

In [33]:
aal_name = "American Airlines Inc."

## 4-2 filtering flight data for AAL

In [34]:
aal_flight_data = sub_df[sub_df["UNIQUE_CARRIER_NAME"] == aal_name]
aal_flight_data

,UNIQUE_CARRIER_NAME,CARRIER_NAME,YEAR,MONTH,QUARTER,PASSENGERS,FREIGHT,MAIL,DISTANCE,PASSENGERS_x_DISTANCE,FREIGHT_x_DISTANCE,MAIL_x_DISTANCE
17,American Airlines Inc.,American Airlines Inc.,1992,1,1,0,0,58,806,0,0,46748
83,American Airlines Inc.,American Airlines Inc.,1992,1,1,0,0,1,1377,0,0,1377
106,American Airlines Inc.,American Airlines Inc.,1992,1,1,0,0,179,2593,0,0,464147
131,American Airlines Inc.,American Airlines Inc.,1992,1,1,0,0,1720,508,0,0,873760
153,American Airlines Inc.,American Airlines Inc.,1992,1,1,0,0,32,2624,0,0,83968
...,...,...,...,...,...,...,...,...,...,...,...,...
7541267,American Airlines Inc.,American Airlines Inc.,1990,8,3,48591,2667136,280048,1597,77599827,4259416192,447236656
7541283,American Airlines Inc.,American Airlines Inc.,1990,7,3,50811,529163,359170,1235,62751585,653516305,443574950
7541286,American Airlines Inc.,American Airlines Inc.,1990,7,3,51443,1222061,88588,1597,82154471,1951631417,141475036
7541293,American Airlines Inc.,American Airlines Inc.,1990,8,3,52959,1485491,92633,1597,84575523,2372329127,147934901


In [35]:
aal_flight_data = aal_flight_data.groupby(["YEAR", "MONTH"]).sum()
aal_flight_data

QUARTER  PASSENGERS   FREIGHT      MAIL  DISTANCE  \
YEAR MONTH                                                      
1990 1         2214     5459701  61310051  37564532   2956474   
     2         1816     5281852  63695856  33957326   2476179   
     3         1867     6370406  74672204  39334963   2598479   
     4         3748     6038262  67249890  36707192   2535670   
     5         3524     6220390  73844411  37455170   2370272   
...             ...         ...       ...       ...       ...   
2019 6         2524    13842231  97970599  15870824   2056794   
     7         3723    14360994  95890553  15043165   2060404   
     8         3693    13935338  93633812  15626078   2024716   
     9         3531    12213890  92596275  15838750   1890587   
     10        4616    13084459  94725433  17946246   1820284   

            PASSENGERS_x_DISTANCE  FREIGHT_x_DISTANCE  MAIL_x_DISTANCE  
YEAR MONTH                                                              
1990 1                 5647500951        111131479478      45661728708  
     2                 5331198760        118800646025      40668495146  
     3                 6419783815        142685520310      47986100277  
     4                 6138906937        128218559295      44384598972  
     5                 6376112716        138673161040      44527554609  
...                           ...                 ...              ...  
2019 6                19726943285        394058461305      36291446704  
     7                20430836494        386337034340      36215206609  
     8                19805502065        376013599675      37046554146  
     9                17176723577        372552001699      37859680114  
     10               17719474598        381735308527      41690254265  

[358 rows x 8 columns]

In [36]:
aal_flight_data = aal_flight_data.reset_index()
aal_flight_data

,YEAR,MONTH,QUARTER,PASSENGERS,FREIGHT,MAIL,DISTANCE,PASSENGERS_x_DISTANCE,FREIGHT_x_DISTANCE,MAIL_x_DISTANCE
0,1990,1,2214,5459701,61310051,37564532,2956474,5647500951,111131479478,45661728708
1,1990,2,1816,5281852,63695856,33957326,2476179,5331198760,118800646025,40668495146
2,1990,3,1867,6370406,74672204,39334963,2598479,6419783815,142685520310,47986100277
3,1990,4,3748,6038262,67249890,36707192,2535670,6138906937,128218559295,44384598972
4,1990,5,3524,6220390,73844411,37455170,2370272,6376112716,138673161040,44527554609
...,...,...,...,...,...,...,...,...,...,...
353,2019,6,2524,13842231,97970599,15870824,2056794,19726943285,394058461305,36291446704
354,2019,7,3723,14360994,95890553,15043165,2060404,20430836494,386337034340,36215206609
355,2019,8,3693,13935338,93633812,15626078,2024716,19805502065,376013599675,37046554146
356,2019,9,3531,12213890,92596275,15838750,1890587,17176723577,372552001699,37859680114


## 4-3 Merging flight data and stock data

In [37]:
final_data = pd.merge(aal_needed, aal_flight_data, left_on=["Year", "Month"], right_on=["YEAR", "MONTH"])
final_data

,Close,Year,Month,YEAR,MONTH,QUARTER,PASSENGERS,FREIGHT,MAIL,DISTANCE,PASSENGERS_x_DISTANCE,FREIGHT_x_DISTANCE,MAIL_x_DISTANCE
0,19.806999,2005,9,2005,9,3333,7465340,100145391,16304050,1784674,10730026141,336721355779,34428612482
1,23.266859,2005,10,2005,10,5336,7717945,105415037,18430564,2135196,10869985007,358375134408,37966613871
2,31.666681,2005,11,2005,11,4500,7851009,100207413,22152726,1805729,10830088383,334101607214,45205804531
3,35.013420,2005,12,2005,12,5672,8171851,93821065,24883054,2255722,11480591657,318802641282,53693160953
4,27.660032,2006,1,2006,1,1125,7794400,86743364,23283118,1835041,11026999588,289482852656,43838669144
...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,32.271446,2019,6,2019,6,2524,13842231,97970599,15870824,2056794,19726943285,394058461305,36291446704
166,30.193247,2019,7,2019,7,3723,14360994,95890553,15043165,2060404,20430836494,386337034340,36215206609
167,26.036852,2019,8,2019,8,3693,13935338,93633812,15626078,2024716,19805502065,376013599675,37046554146
168,26.783880,2019,9,2019,9,3531,12213890,92596275,15838750,1890587,17176723577,372552001699,37859680114


In [38]:
final_data["Date"] = pd.to_datetime(final_data["YEAR"].astype("str") + "-" + final_data["MONTH"].astype("str") + "-01", infer_datetime_format=True)

final_data

,Close,Year,Month,YEAR,MONTH,QUARTER,PASSENGERS,FREIGHT,MAIL,DISTANCE,PASSENGERS_x_DISTANCE,FREIGHT_x_DISTANCE,MAIL_x_DISTANCE,Date
0,19.806999,2005,9,2005,9,3333,7465340,100145391,16304050,1784674,10730026141,336721355779,34428612482,2005-09-01
1,23.266859,2005,10,2005,10,5336,7717945,105415037,18430564,2135196,10869985007,358375134408,37966613871,2005-10-01
2,31.666681,2005,11,2005,11,4500,7851009,100207413,22152726,1805729,10830088383,334101607214,45205804531,2005-11-01
3,35.013420,2005,12,2005,12,5672,8171851,93821065,24883054,2255722,11480591657,318802641282,53693160953,2005-12-01
4,27.660032,2006,1,2006,1,1125,7794400,86743364,23283118,1835041,11026999588,289482852656,43838669144,2006-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,32.271446,2019,6,2019,6,2524,13842231,97970599,15870824,2056794,19726943285,394058461305,36291446704,2019-06-01
166,30.193247,2019,7,2019,7,3723,14360994,95890553,15043165,2060404,20430836494,386337034340,36215206609,2019-07-01
167,26.036852,2019,8,2019,8,3693,13935338,93633812,15626078,2024716,19805502065,376013599675,37046554146,2019-08-01
168,26.783880,2019,9,2019,9,3531,12213890,92596275,15838750,1890587,17176723577,372552001699,37859680114,2019-09-01


In [39]:
final_data_needed = final_data[["Date", "PASSENGERS_x_DISTANCE", "FREIGHT_x_DISTANCE", "MAIL_x_DISTANCE", "Close"]]
final_data_needed = final_data_needed.sort_values("Date", ascending=True)
final_data_needed

,Date,PASSENGERS_x_DISTANCE,FREIGHT_x_DISTANCE,MAIL_x_DISTANCE,Close
0,2005-09-01,10730026141,336721355779,34428612482,19.806999
1,2005-10-01,10869985007,358375134408,37966613871,23.266859
2,2005-11-01,10830088383,334101607214,45205804531,31.666681
3,2005-12-01,11480591657,318802641282,53693160953,35.013420
4,2006-01-01,11026999588,289482852656,43838669144,27.660032
...,...,...,...,...,...
165,2019-06-01,19726943285,394058461305,36291446704,32.271446
166,2019-07-01,20430836494,386337034340,36215206609,30.193247
167,2019-08-01,19805502065,376013599675,37046554146,26.036852
168,2019-09-01,17176723577,372552001699,37859680114,26.783880


## 4-4 fitting a regression line

In [40]:
# final_data_needed["Date"] < test_start_date

In [41]:
from sklearn.linear_model import LinearRegression

target = "Close"
test_start_date = datetime(2018, 1, 1)
train = final_data_needed[final_data_needed["Date"] < test_start_date]
test = final_data_needed[final_data_needed["Date"] > test_start_date]

train_date = train["Date"]
test_date = test["Date"]

train = train.drop("Date", axis=1)
test = test.drop("Date", axis=1)

X_train = train.drop(target, axis=1)
y_train = train[target]

X_test = test.drop(target, axis=1)
y_test = test[target]

In [42]:
lr = LinearRegression()

lr.fit(X_train, y_train)
# y_train.dtypes

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [43]:
y_pred = lr.predict(X_test)

In [44]:
# from matplotlib import pyplot as plt

# fix, ax = plt.subplots()
# ax.plot(test_date, y_test)
# ax.plot(test_date, y_pred)

# ax.legend()
# # ax.plot(y_test)
# # ax.plot(final_data_needed["Date"], final_data_needed["Close"])

# p = figure(plot_width=400, plot_height=400)
# p.circle('petal_length', 'petal_width', source=df)
# show(p)


fig = go.Figure()

show_df = pd.DataFrame({"y test": y_test.values, "y pred": y_pred}, index=test_date.values)
fig.add_trace(go.Scatter(x=show_df.index, y=show_df["y test"], name='fucking fuck'))
fig.add_trace(go.Scatter(x=show_df.index, y=show_df["y pred"], name='fucking fuck'))

# fig.add_trace() = px.line(x=show_df.index, y=show_df["y pred"], title='fucking fuck')
fig.show()
# y_test

# y_pred.shape

In [45]:


X_all = pd.concat([X_train, X_test])
y_all = pd.concat([y_train, y_test])
date_all = pd.concat([train_date, test_date])

lr2 = LinearRegression()
lr2.fit(X_all, y_all)
y_pred_all = lr2.predict(X_all)

fig = go.Figure()

# show_df = pd.DataFrame({"y test": y_test.values, "y pred": y_pred}, index=test_date.values)
fig.add_trace(go.Scatter(x=date_all, y=y_pred_all, name='y_pred_all'))
fig.add_trace(go.Scatter(x=date_all, y=y_all, name='y real'))

fig.show()

# 5- Modeling for all publicly traded airlines

## 5-1 Getting multiple stocks' data and cleaning the data

In [46]:
list_of_publicly_traded_airlines = {
    "UAL": "United Air Lines Inc.",
    "ALK": "Alaska Airlines Inc.",
    "DAL": "Delta Air Lines Inc.",
    "SAVE": "Spirit Air Lines",
    "ALGT": "Allegiant Air",
    "AAL": "American Airlines Inc.",
    "HA": "Hawaiian Airlines Inc.",
    "JBLU": "JetBlue Airways",
    "MESA": "Mesa Airlines Inc.",
    "RYAAY": "Ryan International Airlines",
    "SKYW": "SkyWest Airlines Inc.",
}

In [47]:
data_dict = {}
for symbol in list_of_publicly_traded_airlines.keys():
    data_dict[symbol] = yf.download(
    tickers=symbol,
    period="max",
    interval="1mo",
    auto_adjust=True,
    threads=True,
    actions=True
)   

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [48]:
# checkng for null values

# for symbol in data_dict.values():
#     display(data_df_symbol.shape)
#     display(data_df_symbol.isna().sum())

In [49]:
# removing null values
for symbol in data_dict.keys():
    data_dict[symbol] = data_dict[symbol].dropna(axis=0)


In [50]:
for symbol in data_dict.keys():
    data_dict[symbol]["Year"] = data_dict[symbol].index.year
    data_dict[symbol]["Month"] = data_dict[symbol].index.month
    data_dict[symbol] = data_dict[symbol][["Close", "Year", "Month"]]

In [51]:
# sub_df[sub_df["UNIQUE_CARRIER_NAME"].str.contains("Sky")]["UNIQUE_CARRIER_NAME"].unique()

In [52]:
# ddd

## 5-2 filtering flight data for each symbol

In [53]:
all_flight_data = {}

for symbol in data_dict.keys():
    company_name = list_of_publicly_traded_airlines[symbol]
    all_flight_data[symbol] = sub_df[sub_df["UNIQUE_CARRIER_NAME"] == company_name]


In [54]:
for symbol in data_dict.keys():
    all_flight_data[symbol] = all_flight_data[symbol].groupby(["YEAR", "MONTH"]).sum()

In [55]:
for symbol in data_dict.keys():
    all_flight_data[symbol] = all_flight_data[symbol].reset_index()

## 5-3 Merging flight data and stock data

In [56]:
# merging
final_data_dict = {}
for symbol in data_dict.keys():
    final_data_dict[symbol] = pd.merge(data_dict[symbol], all_flight_data[symbol], left_on=["Year", "Month"], right_on=["YEAR", "MONTH"])

In [57]:
#creating Date column (datetime)
for symbol in data_dict.keys():
    final_data_dict[symbol]["Date"] = pd.to_datetime(final_data_dict[symbol]["YEAR"].astype("str") + "-" + final_data_dict[symbol]["MONTH"].astype("str") + "-01", infer_datetime_format=True)

In [58]:
# taking only the needed columns and sorting by Data
for symbol in data_dict.keys():
    final_data_dict[symbol] = final_data_dict[symbol][["Date", "PASSENGERS_x_DISTANCE", "FREIGHT_x_DISTANCE", "MAIL_x_DISTANCE", "Close"]]
    final_data_dict[symbol] = final_data_dict[symbol].sort_values("Date", ascending=True)

## 5-4 fitting regression lines

In [59]:
# defining a fuction for splitting a time-series data
def time_series_splitter(df, fraction_of_test):
    data_size = len(df)
    test_size = int(data_size * fraction_of_test)
    test_df = df.iloc[-test_size:]
    train_df = df.drop(test_df.index, axis=0)
    if len(train_df) + len(test_df) != data_size:
        st()
    # assert len(train) + len(test_df) == data_size
    return train_df, test_df

In [78]:
linear_regression_models_df = {}
y_test_y_pred_df_dict_linear_regression = {}
target = "Close"


for symbol in final_data_dict.keys():
    data_tmp = final_data_dict[symbol]
    
    train, test = time_series_splitter(data_tmp, 0.2)
    
    train_date = train["Date"]
    test_date = test["Date"]

    train = train.drop("Date", axis=1)
    test = test.drop("Date", axis=1)

    X_train = train.drop(target, axis=1)
    y_train = train[target]

    X_test = test.drop(target, axis=1)
    y_test = test[target]

    lr_tmp = LinearRegression()
    lr_tmp.fit(X_train, y_train)

    y_pred = lr_tmp.predict(X_test)

    fig = go.Figure()
    show_df = pd.DataFrame({"y test": y_test.values, "y pred": y_pred}, index=test_date.values)
    y_test_y_pred_df_dict_linear_regression[symbol] = show_df

    fig.add_trace(go.Scatter(x=show_df.index, y=show_df["y test"], name='predicted'))
    fig.add_trace(go.Scatter(x=show_df.index, y=show_df["y pred"], name='real'))

    linear_regression_models_df[symbol] = lr_tmp
    # fig.add_trace() = px.line(x=show_df.index, y=show_df["y pred"], title='fucking fuck')
    fig.show()

save_object(linear_regression_models_df, "linear_regression_models_df.pickl")
save_object(y_test_y_pred_df_dict_linear_regression, "y_test_y_pred_df_dict_linear_regression.pickl")

# 6- Fitting a Random Forest

In [84]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


random_forest_models_df = {}
y_test_y_pred_df_dict_random_forest = {}
target = "Close"


for symbol in final_data_dict.keys():
    data_tmp = final_data_dict[symbol]
    
    train, test = time_series_splitter(data_tmp, 0.2)
    
    train_date = train["Date"]
    test_date = test["Date"]

    train = train.drop("Date", axis=1)
    test = test.drop("Date", axis=1)

    X_train = train.drop(target, axis=1)
    y_train = train[target]

    X_test = test.drop(target, axis=1)
    y_test = test[target]

    pipeline = make_pipeline(
        RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=0)
    )

    param_distributions = {
        'randomforestregressor__max_depth': list(range(2, 25)), 
    }
    rf_tmp = GridSearchCV(
        pipeline,
        param_grid=param_distributions,
        cv=5,
        scoring='neg_mean_absolute_error',
        verbose=10,
        return_train_score=True,
        n_jobs=-1
    )

    rf_tmp.fit(X_train, y_train)

    y_pred = rf_tmp.predict(X_test)

    fig = go.Figure()
    show_df = pd.DataFrame({"y test": y_test.values, "y pred": y_pred}, index=test_date.values)
    y_test_y_pred_df_dict_random_forest[symbol] = show_df

    fig.add_trace(go.Scatter(x=show_df.index, y=show_df["y test"], name='predicted'))
    fig.add_trace(go.Scatter(x=show_df.index, y=show_df["y pred"], name='real'))

    random_forest_models_df[symbol] = rf_tmp.best_estimator_
    
    # fig.add_trace() = px.line(x=show_df.index, y=show_df["y pred"], title='fucking fuck')
    # fig.show()

save_object(y_test_y_pred_df_dict_random_forest, "y_test_y_pred_df_dict_random_forest.pickl")
save_object(random_forest_models_df, "random_forest_models_df.pickl")


TypeError: __init__() got an unexpected keyword argument 'n_iter'

In [80]:
# random_forest_models_df

for symbol in final_data_dict.keys():
    model = random_forest_models_df[symbol]
    # st()
    print(max([estimator.tree_.max_depth for estimator in model.estimators_]))

19
26
16
15
16
24
22
17
7
21
22
